# Empire Monkey
Detects EmpireMonkey APT reported Activity

## Rule Content
```
- action: global
  title: Empire Monkey
  id: 10152a7b-b566-438f-a33c-390b607d1c8d
  description: Detects EmpireMonkey APT reported Activity
  references:
  - https://app.any.run/tasks/a4107649-8cb0-41af-ad75-113152d4d57b
  tags:
  - attack.t1086
  - attack.execution
  date: 2019/04/02
  author: Markus Neis
  detection:
    condition: 1 of them
  falsepositives:
  - Very Unlikely
  level: critical
- logsource:
    category: process_creation
    product: windows
  detection:
    selection_cutil:
      CommandLine:
      - '*/i:%APPDATA%\logs.txt scrobj.dll'
      Image:
      - '*\cutil.exe'
    selection_regsvr32:
      CommandLine:
      - '*/i:%APPDATA%\logs.txt scrobj.dll'
      Description:
      - Microsoft(C) Registerserver

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_command_line.keyword:(*\/i\:%APPDATA%\\logs.txt\ scrobj.dll) AND (process_path.keyword:(*\\cutil.exe) OR file_description:("Microsoft\(C\)\ Registerserver")))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()